# OpenVINO (50 баллов)

## Установим Зависимости

In [ ]:
# !pip install -U openvino nncf
# # или пре-релизная версия:
# !pip install --pre -U openvino --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly git+https://github.com/openvinotoolkit/nncf.git

# !pip install transformers[torch] datasets evaluate

## Скачиваем Предобученную Модель

Выберите классификационную модель из [Huggingface Hub](https://huggingface.co/models?pipeline_tag=text-classification&sort=trending&search=sst2), либо возьмите модель по умолчанию. Этот ноутбук сделан с рассчётом модель, натренированную на [sst2](https://huggingface.co/datasets/nyu-mll/glue/viewer/sst2) датасете. Если выберете другую модель и датасет, перепешите соответствующие блоки проверки accuracy. Единственное ограничение - модель должна быть трансформер энкодером.

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_id = "philschmid/MiniLM-L6-H384-uncased-sst2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_model = AutoModelForSequenceClassification.from_pretrained(model_id)

## Конвертируем Модель в OpenVINO (5 баллов)



In [2]:
import openvino as ov


example_input = {**tokenizer("test", return_tensors="pt")}
ov_model = ov.convert_model(hf_model, example_input=example_input)
ov.save_model(ov_model, 'bert.xml')

compiled_model = ov.compile_model(ov_model)
print(compiled_model(example_input))

/home/apaniuko/python/deepschool/venv/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


{<ConstOutput: names[logits] shape[?,2] type: f32>: array([[ 1.1164335, -1.1053413]], dtype=float32)}


Провалидируем предсказания сконвертированной модели:

In [54]:
import torch
import numpy as np
from datasets import load_dataset
import evaluate


val_dataset = load_dataset("glue", "sst2", split="validation")
accuracy = evaluate.load("accuracy")


@torch.no_grad
def accuracy_evaluate(model, dataset=val_dataset, accuracy=accuracy):   
    for sample in dataset:
        tokenized = {**tokenizer(sample["sentence"], return_tensors="pt")}
        logits = model(tokenized)["logits"]
        pred = np.argmax(logits, axis=1)
        accuracy.add(references=sample["label"], predictions=pred)

    return accuracy.compute()["accuracy"]


print(f"PyTorch accuracy:  {accuracy_evaluate(lambda x: hf_model(**x))}")
print(f"OpenVINO accuracy: {accuracy_evaluate(compiled_model)}")

PyTorch accuracy:  0.9013761467889908
OpenVINO accuracy: 0.9013761467889908


## Benchmark (5 баллов)

Добавьте несколько инференсов в бенчмарк для того, чтобы получить более точные результаты.

In [4]:
from time import perf_counter
from statistics import median


@torch.no_grad
def benchmark(model, dataset):
    tokenized_dataset = [{**tokenizer(sample["sentence"], return_tensors="pt")} for sample in dataset]

    # warmup
    for data in tokenized_dataset[:10]:
        model(data)
    
    times = []
    for data in tokenized_dataset:
        start = perf_counter()
        model(data)
        end = perf_counter()
        times.append(end - start)

    return (
        f"{sum(times):.5f}s, FPS={(len(dataset) / sum(times)):.3f}, "
        f"latency: {min(times):.5f}s, {median(times):.5f}s, {max(times):.5f}s"
    )

print("Pytorch:  ", benchmark(lambda x: hf_model(**x), val_dataset))
print("Openvino: ", benchmark(lambda x: compiled_model(x), val_dataset))

Pytorch:   5.31208s, FPS=164.154, latency: 0.00459s, 0.00605s, 0.01031s
Openvino:  2.45941s, FPS=354.556, latency: 0.00207s, 0.00280s, 0.00367s


## Inference Hints (3 балла)

Скомпилируйте модель с разными инференс хинтами и сравните результаты бенчмарка. Не забудьте указать "CPU" в качестве таргета.

In [5]:
import openvino.properties as props
import openvino.properties.hint as hints

compiled_througput = ov.compile_model(
    ov_model, 
    "CPU", 
    {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
)
print("Openvino: ", benchmark(lambda x: compiled_througput(x), val_dataset))

Openvino:  3.34922s, FPS=260.359, latency: 0.00248s, 0.00374s, 0.00744s


In [6]:
compiled_latency = ov.compile_model(
    ov_model, 
    "CPU", 
    {hints.performance_mode: hints.PerformanceMode.LATENCY}
)
print("Openvino: ", benchmark(lambda x: compiled_latency(x), val_dataset))

Openvino:  2.43760s, FPS=357.728, latency: 0.00203s, 0.00275s, 0.00505s


## Async Inference

Переписать бенчмарк под асинхронный инференс. Он должен принимать на вход асинхронную очередь и датасет.

### Простой Бенчмарк (5 баллов)
Простая версия бенчмарка должна замерить FPS:

In [7]:
from typing import Dict, Any, Optional


def completion_callback(
    infer_request: ov.InferRequest, user_data: Optional[Dict[str, Any]] = None
) -> None:
    ...  # в простом бенчмарке информация из реквеста нам не нужна


# benchmark app использует 18 реквестов, очередь без указания jobs создаёт 12
infer_queue = ov.AsyncInferQueue(compiled_througput, jobs=18)
infer_queue.set_callback(completion_callback)


def simple_benchmark_async(queue, dataset):
    tokenized_dataset = [{**tokenizer(sample["sentence"], return_tensors="np")} for sample in dataset]

    # warmup
    for data in tokenized_dataset[:10]:
        queue.start_async(data)
    queue.wait_all()
    
    results = [0 for _ in range(len(dataset))]
    start = perf_counter()
    for idx, data in enumerate(tokenized_dataset):
        queue.start_async(data)
    queue.wait_all()
    end = perf_counter()
    elapsed = end - start

    return f"{elapsed:.5f}s, FPS={(len(dataset) / elapsed):.3f}"


print("Openvino: ", simple_benchmark_async(infer_queue, val_dataset))

Openvino:  0.58401s, FPS=1493.113


### Добавить Измерение latency В Асинхронный Бенчмарк (14 баллов)

Используйте `completion_callback` для подсчёта latency.

In [8]:
def completion_callback(
    infer_request: ov.InferRequest,
    user_data: Dict[str, Any],
) -> None:
    end = perf_counter()  # инференс завершился, заменяем время
    idx = user_data["idx"]
    times = user_data["times"]
    times[idx] = end - times[idx]  # вычитаем время начала 


# используем существующую очередь, переназначим коллбэк
infer_queue.set_callback(completion_callback)


def benchmark_async(queue, dataset):
    tokenized_dataset = [{**tokenizer(sample["sentence"], return_tensors="np")} for sample in dataset]
    times = [0 for _ in range(len(dataset))]

    # warmup
    for data in tokenized_dataset[:10]:
        queue.start_async(data, {"idx": 0, "times": times})
    queue.wait_all()
    
    start = perf_counter()
    for idx, data in enumerate(tokenized_dataset):
        # записываем время старта реквеста в массив по индексу входных данных
        times[idx] = perf_counter()
        # передаём индекс и массив с началами вместе с входными данными
        queue.start_async(data, {"idx": idx, "times": times})
    # ждём пока завершатся все реквесты
    queue.wait_all()

    # замеряем время конца инференса
    end = perf_counter()
    # для общего времени исполнения уже нельзя брать sum(times), так как реквесты исполняются одновременно
    elapsed = end - start
    
    return (
        f"{elapsed:.5f}s, FPS={(len(dataset) / elapsed):.3f}, "
        f"latency: {min(times):.5f}s, {median(times):.5f}s, {max(times):.5f}s"
    )


print("Openvino: ", benchmark_async(infer_queue, val_dataset))

Openvino:  0.55598s, FPS=1568.406, latency: 0.00451s, 0.01018s, 0.03759s


`InferRequest` объект сам замеряет latency во время инференса, поэтому можно просто достать время оттуда. Время замеряется в плюсах, поэтому latency получается немного меньше. benchmark app для замеров latency тоже берёт информацию из реквеста.

In [9]:
def latency_from_ir_completion_callback(
    infer_request: ov.InferRequest,
    user_data: Dict[str, Any],
) -> None:
    times = user_data["times"]
    idx = user_data["idx"]
    times[idx] = infer_request.latency * 1e-3  # ms -> s


infer_queue.set_callback(latency_from_ir_completion_callback)
print("Openvino: ", benchmark_async(infer_queue, val_dataset))

Openvino:  0.54669s, FPS=1595.067, latency: 0.00441s, 0.00960s, 0.03375s


## Benchmark App

### Измерьте Производительность Модели с Помощью CLI benchmark_app (1 балл)

Чтобы не ждать по минуте можете использовать флаг `-t 30`. Выполнение какого слоя занимает больше всего времени?

In [40]:
!benchmark_app -m "bert.xml" -shape [1,128] -t 30 | tail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Step 11/11] Dumping statistics report
[ INFO ] Execution Devices:['CPU']
[ INFO ] Count:            21024 iterations
[ INFO ] Duration:         30029.90 ms
[ INFO ] Latency:
[ INFO ]    Median:        22.94 ms
[ INFO ]    Average:       25.39 ms
[ INFO ]    Min:           13.30 ms
[ INFO ]    Max:           124.35 ms
[ INFO ] Throughput:   700.10 FPS


In [34]:
!mkdir -p benchmark_report
!benchmark_app -m "bert.xml" -shape [1,128] -t 30 -report_folder benchmark_report -pc -pcsort simple_sort \
-report_type average_counters | tail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[ INFO ] Statistics report is stored to benchmark_report/benchmark_report.csv
[ INFO ] Execution Devices:['CPU']
[ INFO ] Count:            21006 iterations
[ INFO ] Duration:         30042.05 ms
[ INFO ] Latency:
[ INFO ]    Median:        22.96 ms
[ INFO ]    Average:       25.36 ms
[ INFO ]    Min:           13.39 ms
[ INFO ]    Max:           99.35 ms
[ INFO ] Throughput:   699.22 FPS


In [39]:
!head -n 3 benchmark_report/benchmark_sorted_report.csv

layerName,execStatus,layerType,execType,realTime (ms),cpuTime (ms),"proportion (%)
"
__module.bert.encoder.layer.5.intermediate.dense/aten::linear/MatMul,Status.EXECUTED,FullyConnected,brgconv_avx512_1x1_f32,1.108,1.108,4.80%


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


> ⚠️ Окончательный замер всегда нужно проводить с отключенными perf_counter'ами, так как сбор такой статистики замедляет инференс

### (Optional) Попробуйте подобрать параметры, чтобы увеличить FPS относительно

In [26]:
import json


with open("ov_config.json", "w") as config_file:
    json.dump(
        {
            "CPU": {
                "NUM_STREAMS": 24,
                "INFERENCE_NUM_THREADS": 48,
            }
        },
        config_file, 
    )

!benchmark_app -m "bert.xml" -shape [1,128] -d CPU -load_config ov_config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.0.0-14509-34caeefd078-releases/2024/0
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.0.0-14509-34caeefd078-releases/2024/0
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 28.82 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     logits (node: __mo

За счёт ухудшения latency удалось немного повысить throughput.

> ⚠️ Такие измерения нужно производить непосредственно на железе, которое будет использоваться для инференса 

## NNCF

### Дефолтная Квантизация (5 баллов)

Квантизуйте модель с дефолтными параметрами. Замерьте accuracy.

In [62]:
import nncf


def transform_fn(text):
    return {**tokenizer(text["sentence"], return_tensors="np")}


# возьмём для калибрации другой датасет
test_dataset = load_dataset("glue", "sst2", split="test[:300]")
calibration_dataset = nncf.Dataset(test_dataset, transform_fn)
quntized_model = nncf.quantize(
    ov_model, 
    calibration_dataset=calibration_dataset,
    preset=nncf.QuantizationPreset.MIXED,
    target_device=nncf.TargetDevice.CPU,  # важно
    model_type=nncf.ModelType.TRANSFORMER,  # очень важно!
)
ov.save_model(quntized_model, "qbert.xml", compress_to_fp16=False)

Output()

Output()

INFO:nncf:18 ignored nodes were found by name in the NNCFGraph
INFO:nncf:26 ignored nodes were found by name in the NNCFGraph


Output()

Output()

In [63]:
compiled_quantized = ov.compile_model(quntized_model, "CPU", {hints.performance_mode: hints.PerformanceMode.THROUGHPUT})
print(f"Quantized OpenVINO accuracy: {accuracy_evaluate(compiled_quantized)}")

Quantized OpenVINO accuracy: 0.9025229357798165


Замерьте FPS квантизованной модели с помощью benchmark функции или benchmark_app:

In [64]:
qinfer_queue = ov.AsyncInferQueue(compiled_quantized, jobs=18)
qinfer_queue.set_callback(completion_callback)

print("Openvino: ", benchmark_async(qinfer_queue, val_dataset))

Openvino:  0.40821s, FPS=2136.181, latency: 0.00313s, 0.00781s, 0.02472s


In [65]:
!benchmark_app -m "qbert.xml" -shape [1,128] -t 30 | tail

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Step 11/11] Dumping statistics report
[ INFO ] Execution Devices:['CPU']
[ INFO ] Count:            44334 iterations
[ INFO ] Duration:         30023.66 ms
[ INFO ] Latency:
[ INFO ]    Median:        10.33 ms
[ INFO ]    Average:       11.60 ms
[ INFO ]    Min:           6.22 ms
[ INFO ]    Max:           60.13 ms
[ INFO ] Throughput:   1476.64 FPS


### Accuracy Control (10 баллов)

Квантизуйте модель так, чтобы потеря accuracy была в пределах 1%. Замерьте FPS получившейся модели.

In [67]:
# разделим валидационный датасет для финальной валидации
validation_dataset = list(val_dataset)
final_test_dataset, validation_dataset = validation_dataset[:-300], validation_dataset[-300:]
validation_dataset = nncf.Dataset(validation_dataset, transform_fn)

quntized_model = nncf.quantize_with_accuracy_control(
    model=ov_model,
    calibration_dataset=calibration_dataset,
    preset=nncf.QuantizationPreset.PERFORMANCE,
    validation_dataset=validation_dataset,
    validation_fn=accuracy_evaluate,  # функция замера accuracy переиспользуется
    max_drop=0.01,
    target_device=nncf.TargetDevice.CPU,
    drop_type=nncf.DropType.ABSOLUTE,
    # уберём тип модели, чтобы получить accuracy drop больше 1%
    # иначе квантизация будет совпадать с дефолтной 🤷
    # model_type=nncf.ModelType.TRANSFORMER, 
)
ov.save_model(quntized_model, "qbert_acc.xml", compress_to_fp16=False)

Output()

Output()

INFO:nncf:Validation of initial model was started
INFO:nncf:Elapsed Time: 00:00:00
INFO:nncf:Elapsed Time: 00:00:01
INFO:nncf:Metric of initial model: 0.8966666666666666
INFO:nncf:Collecting values for each data item using the initial model
INFO:nncf:Elapsed Time: 00:00:03
INFO:nncf:Validation of quantized model was started
INFO:nncf:Elapsed Time: 00:00:00
INFO:nncf:Elapsed Time: 00:00:01
INFO:nncf:Metric of quantized model: 0.85
INFO:nncf:Collecting values for each data item using the quantized model
INFO:nncf:Elapsed Time: 00:00:04
INFO:nncf:Accuracy drop: 0.046666666666666634 (absolute)
INFO:nncf:Accuracy drop: 0.046666666666666634 (absolute)
INFO:nncf:Total number of quantized operations in the model: 85
INFO:nncf:Number of parallel workers to rank quantized operations: 1
INFO:nncf:ORIGINAL metric is used to rank quantizers


Output()

INFO:nncf:Elapsed Time: 00:03:00
INFO:nncf:Changing the scope of quantizer nodes was started
INFO:nncf:Reverted 4 operations to the floating-point precision: 
	__module.bert.encoder.layer.0.attention.self.key/aten::linear/MatMul
	__module.bert.encoder.layer.0.attention.output/aten::add/Add
	__module.bert.encoder.layer.0.attention.self.value/aten::linear/MatMul
	__module.bert.encoder.layer.0.attention.self.query/aten::linear/MatMul
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.0.output.LayerNorm/aten::layer_norm/MVN
INFO:nncf:Accuracy drop with the new quantization scope is 0.023333333333333317 (absolute)
INFO:nncf:Re-calculating ranking scores for remaining groups


...

INFO:nncf:Elapsed Time: 00:03:13
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert/aten::rsub/Multiply
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert/aten::rsub/Subtract
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.4.attention.self/aten::matmul/MatMul_1
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.5.attention.self/aten::matmul/MatMul
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.5.attention.self/aten::matmul/MatMul_1
I

Output()

INFO:nncf:Elapsed Time: 00:02:44
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.5.attention.output.LayerNorm/aten::layer_norm/MVN
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.4.output.LayerNorm/aten::layer_norm/MVN
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 2 operations to the floating-point precision: 
	__module.bert.encoder.layer.0.intermediate.dense/aten::linear/MatMul
	__module.bert.encoder.layer.0.output/aten::add/Add
INFO:nncf:Accuracy drop with the new quantization scope is 0.016666666666666607 (absolute)
INFO:nncf:Reverted 4 operations to the floating-point precision: 
	__module.bert.encoder.layer.4.attention.output/aten::add/Add
	__module.bert.encoder.layer.4.attention.self.value/aten::linear/MatMul
	__module.bert.encoder.lay

Output()

INFO:nncf:Elapsed Time: 00:02:42
INFO:nncf:Reverted 1 operations to the floating-point precision: 
	__module.bert.encoder.layer.2.attention.self/aten::matmul/MatMul
INFO:nncf:Algorithm completed: achieved required accuracy drop 0.009999999999999898 (absolute)
INFO:nncf:19 out of 85 were reverted back to the floating-point precision:
	__module.bert.encoder.layer.0.attention.self.key/aten::linear/MatMul
	__module.bert.encoder.layer.0.attention.output/aten::add/Add
	__module.bert.encoder.layer.0.attention.self.value/aten::linear/MatMul
	__module.bert.encoder.layer.0.attention.self.query/aten::linear/MatMul
	__module.bert/aten::rsub/Multiply
	__module.bert/aten::rsub/Subtract
	__module.bert.encoder.layer.4.attention.self/aten::matmul/MatMul_1
	__module.bert.encoder.layer.5.attention.self/aten::matmul/MatMul
	__module.bert.encoder.layer.5.attention.self/aten::matmul/MatMul_1
	__module.bert.encoder.layer.5.attention.output.dense/aten::linear/MatMul
	__module.bert.encoder.layer.5.output.dense

In [68]:
compiled_quantized_acc = ov.compile_model(
    "qbert_acc.xml", "CPU", {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
)

print(f"Openvino: {accuracy_evaluate(compiled_througput, final_test_dataset)}")
print(f"Quantized Openvino :  {accuracy_evaluate(compiled_quantized, final_test_dataset)}")
print(f"Openvino quantized with acc:  {accuracy_evaluate(compiled_quantized_acc, final_test_dataset)}")

Openvino: 0.9038461538461539
Quantized Openvino :  0.9038461538461539
Openvino quantized with acc:  0.8706293706293706


Как видно из результатов на тестовом датасете:
- Определённый дроп на валидационном датасете ничего не гарантирует.
- Указать тип модели бывает важнее, чем дать валидационный датасет.

# Дополнительно (2 балла)

Добавьте в модель:
1. Токенизационный препроцессинг с помощью `openvino-tokenizers`
2. (Hard) Добавьте постпроцессинг в модель, чтобы она сразу отдавала результат `np.argmax(logits, axis=1)`

In [ ]:
# !pip install --pre -U openvino-tokenizers --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly git+https://github.com/openvinotoolkit/nncf.git

In [69]:
from openvino_tokenizers import convert_tokenizer, connect_models


def get_connected_model(hf_model, tokenizer):
    example_input = {**tokenizer("test", return_tensors="pt")}
    ov_model = ov.convert_model(hf_model, example_input=example_input)
    ov_tokenizer = convert_tokenizer(tokenizer)
    return connect_models(ov_tokenizer, ov_model)


get_connected_model(hf_model, tokenizer)

/home/apaniuko/python/deepschool/venv/lib/python3.10/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
RegexNormalization pattern is not supported, operation output might differ from the original tokenizer.


<Model: 'Model1655_with_Model1653'
inputs[
<ConstOutput: names[Parameter_4126935] shape[?] type: string>
]
outputs[
<ConstOutput: names[logits] shape[?,2] type: f32>
]>

### Задание 2

Требуется добавить кастомную операцию после выхода `logits`. В OpenVINO нет операции `argmax`, но есть [TopK](https://docs.openvino.ai/2024/documentation/openvino-ir-format/operation-sets/operation-specs/sort/top-k-11.html), которая возвращает значения и индексы топ К максимальных элементов - нам нужен второй выход при `k=1`.

Воспользуемся [примером](https://docs.openvino.ai/2022.3/openvino_docs_OV_UG_Preprocessing_Details.html#custom-operations) из документации:

In [70]:
from openvino.preprocess import PrePostProcessor
from openvino.runtime import opset14 as opset, Output
from openvino.runtime.utils.decorators import custom_preprocess_function


@custom_preprocess_function
def custom_argmax_1(output: Output):
    argmax = opset.topk(output, k=1, axis=1, mode="max", sort="none")
    return argmax


connected_model = get_connected_model(hf_model, tokenizer)
ppp = PrePostProcessor(connected_model)
ppp.output("logits").postprocess().custom(custom_argmax_1)
connected_model = ppp.build()

RegexNormalization pattern is not supported, operation output might differ from the original tokenizer.


RuntimeError: Check 'false' failed at src/core/src/node.cpp:148:
While validating node 'opset11::TopK TopK_4133347 (opset1::Add __module.classifier/aten::linear/Add[0]:f32[?,2], opset1::Constant Constant_4133346[0]:i64[]) -> (f32[?,1], i32[?,1])' with friendly_name 'TopK_4133347':
Default output not supported


Проблема в том, что у PPP есть ограничение - только один вход и один выход. Попробуем вернуть нужный нам выход:

In [71]:
@custom_preprocess_function
def custom_argmax_2(output: Output):
    argmax = opset.topk(output, k=1, axis=1, mode="max", sort="none")
    return argmax.output(1)


connected_model = get_connected_model(hf_model, tokenizer)
ppp = PrePostProcessor(connected_model)
ppp.output("logits").postprocess().custom(custom_argmax_2)
connected_model = ppp.build()

RegexNormalization pattern is not supported, operation output might differ from the original tokenizer.


TypeError: _from_node(): incompatible function arguments. The following argument types are supported:
    1. (self: openvino._pyopenvino.Node) -> openvino._pyopenvino.Output

Invoked with: <Output: names[] shape[?,1] type: i32>

И тут мимо. Но... стектрейс позволяет нам заглянуть во внутренности `custom_preprocess_function`:
```python
     87 @wraps(custom_function)
     88 def wrapper(node: Node) -> Output:
---> 89     return Output._from_node(custom_function(node))
```
Она берёт ноду (которую мы возвращали в `custom_argmax_1`) и пытается получить её выход. Поэтому мы можем убрать декоратор и вернуть нужный выход самостоятельно:

In [72]:
def custom_argmax_3(output: Output):
    argmax = opset.topk(output, k=1, axis=1, mode="max", sort="none")
    return argmax.output(1)


connected_model = get_connected_model(hf_model, tokenizer)
ppp = PrePostProcessor(connected_model)
ppp.output("logits").postprocess().custom(custom_argmax_3)
connected_model_wiht_argmax_1 = ppp.build()

compiled_connected = ov.compile_model(connected_model_wiht_argmax_1)
compiled_connected(["Test", "Something completely different"])

RegexNormalization pattern is not supported, operation output might differ from the original tokenizer.
/tmp/tmpctejxcqg/build/third_party/re2/src/extern_re2/re2/re2.cc:205: Error parsing '((?=[^\n\t\r])\p{Cc})|((?=[^\n\t\r])\p{Cf})': invalid perl operator: (?=


{<ConstOutput: names[logits] shape[?,1] type: i32>: array([[0],
       [1]], dtype=int32)}

Такая кастомная операция оставляет "лишнюю" размерность. Её убрать с помощью операции [Squeeze](https://docs.openvino.ai/2024/documentation/openvino-ir-format/operation-sets/operation-specs/shape/squeeze-1.html). У этой операции один выход, поэтому мы можем вернуть `custom_preprocess_function` декоратор:

In [73]:
@custom_preprocess_function
def custom_argmax_4(output: Output):
    argmax = opset.topk(output, k=1, axis=1, mode="max", sort="none")
    squeeze = opset.squeeze(
        data=argmax.output(1),
        axes=1,
    )
    return squeeze

connected_model = get_connected_model(hf_model, tokenizer)
ppp = PrePostProcessor(connected_model)
ppp.output("logits").postprocess().custom(custom_argmax_4)
connected_model_wiht_argmax_2 = ppp.build()

# поменяем имя выхода
connected_model_wiht_argmax_2.output().tensor.set_names({"class_label"})

compiled_connected = ov.compile_model(connected_model_wiht_argmax_2)
compiled_connected(["Test", "Something completely different"])

RegexNormalization pattern is not supported, operation output might differ from the original tokenizer.
/tmp/tmpctejxcqg/build/third_party/re2/src/extern_re2/re2/re2.cc:205: Error parsing '((?=[^\n\t\r])\p{Cc})|((?=[^\n\t\r])\p{Cf})': invalid perl operator: (?=


{<ConstOutput: names[class_label] shape[?] type: i32>: array([0, 1], dtype=int32)}

In [74]:
@torch.no_grad
def accuracy_evaluate_for_connected_model(model, dataset=val_dataset, accuracy=accuracy):   
    for sample in dataset:
        pred = model([sample["sentence"]])["class_label"]
        accuracy.add(references=sample["label"], predictions=pred)

    return accuracy.compute()["accuracy"]

print(f"Final model accuracy: {accuracy_evaluate_for_connected_model(compiled_connected)}")

Final model accuracy: 0.9013761467889908
